In [113]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

In [148]:
def string_to_bin(mensagem: str) -> list:
    ''' Essa função recebe como entrada uma string
    e devolve os binarios 0s e 1s que a representam.
    Essa função pode receber letra por letra ou uma 
    string inteira e converter na sequencia binaria
    que a representa.

    Args:
            - mensagem (`string`): mensagem que deseja obter o binario.
    Returns:
            - (`list`): lista que representa a sequencia binaria do string.
    '''
    return [format(ord(c), '08b') for c in mensagem]


def bin_to_string(bin: list) -> str:
    ''' Essa função recebe uma sequencia de binarios e converte em uma
    string.

    Args:
            - bin (`list`): lista contendo os binarios.
    Returns:
            - (`string`): string da mensagem decodificada.
    '''
    return ''.join([chr(int(b, 2)) for b in bin])

In [157]:
def split_into_pairs(bin: list) -> list:
    result = []

    for byte in bin:
        pairs = [byte[i:i+2] for i in range(0, len(byte), 2)]
        result.append(pairs)
    
    return result


def list_from_pairs(pairs: list) -> list:
    byte_list = []

    for pair in pairs:
        byte = ''.join(pair)
        byte_list.append(byte)

    return byte_list

In [162]:
def emaranhar(qubits: int) -> QuantumCircuit:
    '''Aqui, você deve construir um circuito que emaranha o seu qubit 
    com o de Alice.

    Args:
        - int (int): Recebe o tamanho do circuito compartilhado com 
            Alice.
    Returns:
        - (QuantumCircuit): Circuito compartilhado entre voce e Alice.
    '''

    qc = QuantumCircuit(qubits)

    qc.h(0)
    qc.cx(0, 1)

    return qc


def sua_parte(qc: QuantumCircuit, bin: list) -> QuantumCircuit:
    ''' Aqui você aplicará as rotações necessárias dependendo dos bits que deseja
    enviar para Alice.

    Args:
        - qc (`QuantumCircuit`): Recebe o circuito compartilhado por
            voce e alice.
        - bin (list): Recebe os binarios que deseja enviar por meio de rotações.
    Returns:
        - (QuantumCircuit): Circuito compartilhado entre voce e Alice.
    '''

    if bin[0] == '1':
        qc.z(0)
    if bin[1] == '1':
        qc.x(0)

    return qc


def alice_parte(qc: QuantumCircuit) -> list:
    ''' Alice recebe esses qubits, mede eles e armazena os bits em um vetor.
    Args:
        - qc (QuantumCircuit): Recebe o circuito compartilhado por
            voce e alice.
    Returns:
        - (list): lista com os binarios medidos codificados.
    '''

    qc.cx(0, 1)
    qc.h(0)
    qc.measure_all()

    result = AerSimulator().run(qc, shots=1).result()
    stats = result.get_counts()

    return list(stats.keys())

In [171]:
def protocol(bin: list) -> list:
    ''' Essa função realiza o protocolo mostrado em sala. Aqui, você deve
    construir um circuito que emaranha o seu qubit com o de Alice, e dependendo
    dos binarios que você quer enviar, realiza rotações no seu qubit. 

    Logo apos isso, Alice recebe esses qubits, mede eles e armazena os bits em um vetor.
    - Lembre: a cada qubit emaranhado, você pode enviar 2 bits de informação.
    
    Aqui você pode emaranhar apenas um qubits e a partir dele, enviar de 2 em 2 bits de informação,
    ou construir um circuito maior que envia mais bits de informação. 
    
    De qualquer forma, pela limitação de simular um computador quântico, você não podera 
    colocar todos os qubits de uma vez só, e realizar essa função varias vezes pode ser um boa solução.
    
    Args:
        - bin (list): lista contendo os binarios que deseja enviar.
    Returns:
        - (list): lista contendo os binarios enviados.
    '''
    pairs = split_into_pairs(bin)

    result = []
    for byte in pairs:
        byte_result = []
        for pair in byte:
            qc = emaranhar(2)
            qc = sua_parte(qc, list(pair))
            received = alice_parte(qc)
            adjusted = received[0][::-1]
            
            byte_result.append(adjusted)
        result.append(byte_result)

    return result

In [184]:
message = "quantic is cool"
message_bin = string_to_bin(message)

received_message = protocol(message_bin)
decoded_message = bin_to_string(list_from_pairs(received_message))

print(decoded_message)

quantic is cool
